# Simple Feature Engineering Pipeline - CORRECTED
## BDI-II Depression Score Prediction - IEEE EMBS BHI 2025

**CORRECTED APPROACH**: This notebook implements a clean, simple feature engineering pipeline that:
- ✅ Creates binary columns for ALL 4 real medical conditions (including Cancer)
- ✅ Creates proper one-hot encoding for all condition types
- ✅ Uses clear naming conventions to avoid confusion
- ✅ Focuses on clinically meaningful transformations

### Real Medical Conditions (4 total):
1. **Cancer** (64.7% of patients)
2. **Acute coronary syndrome** (23.4% of patients)
3. **Renal insufficiency** (6.0% of patients) 
4. **Lower-limb amputation** (6.0% of patients)

### Condition Types (7 subcategories):
- Breast, Prostate, Revascularization, No prosthesis, Predialysis, Percutaneous coronary intervention, Dialysis

### Key Principle:
**Create proper binary encoding for real medical conditions AND one-hot encoding for condition subtypes**

In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats
import json
from datetime import datetime

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"📅 Corrected Feature Engineering Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🎯 Goal: Create ALL real medical condition columns + proper condition type encoding")

✅ Libraries imported successfully!
📅 Corrected Feature Engineering Started: 2025-10-03 17:26:54
🎯 Goal: Create ALL real medical condition columns + proper condition type encoding


## Step 1: Data Loading & Analysis

In [5]:
# Load and analyze the data
print("📂 STEP 1: Loading and Analyzing Data")
print("="*50)

# Load datasets
train_path = '../Track1_Data/processed/train_split.xlsx'
test_path = '../Track1_Data/processed/test_split.xlsx'

df_train = pd.read_excel(train_path)
df_test = pd.read_excel(test_path)

print(f"📊 Training Data: {df_train.shape}")
print(f"📊 Test Data: {df_test.shape}")

# Analyze medical conditions
print(f"\n🏥 MEDICAL CONDITIONS ANALYSIS:")
print("="*50)

print("1. MAIN CONDITIONS:")
conditions = df_train['condition'].value_counts()
for condition, count in conditions.items():
    pct = (count / len(df_train)) * 100
    print(f"   • {condition}: {count} patients ({pct:.1f}%)")

print("\n2. CONDITION TYPES (Sub-categories):")
condition_types = df_train['condition_type'].value_counts()
for ctype, count in condition_types.items():
    pct = (count / len(df_train)) * 100
    print(f"   • {ctype}: {count} patients ({pct:.1f}%)")

print("\n3. CONDITION TYPE BY MAIN CONDITION:")
cross_tab = pd.crosstab(df_train['condition'], df_train['condition_type'])
display(cross_tab)

# Store the real medical conditions
real_medical_conditions = list(conditions.index)
real_condition_types = list(condition_types.index)

print(f"\n✅ Identified {len(real_medical_conditions)} real medical conditions")
print(f"✅ Identified {len(real_condition_types)} condition subtypes")

📂 STEP 1: Loading and Analyzing Data
📊 Training Data: (167, 10)
📊 Test Data: (43, 10)

🏥 MEDICAL CONDITIONS ANALYSIS:
1. MAIN CONDITIONS:
   • Cancer: 108 patients (64.7%)
   • Acute coronary syndrome: 39 patients (23.4%)
   • Renal insufficiency: 10 patients (6.0%)
   • Lower-limb amputation: 10 patients (6.0%)

2. CONDITION TYPES (Sub-categories):
   • Breast: 67 patients (40.1%)
   • Prostate: 41 patients (24.6%)
   • Revascularization: 31 patients (18.6%)
   • No prosthesis: 10 patients (6.0%)
   • Predialysis: 9 patients (5.4%)
   • Percutaneous coronary intervention: 8 patients (4.8%)
   • Dialysis: 1 patients (0.6%)

3. CONDITION TYPE BY MAIN CONDITION:


condition_type,Breast,Dialysis,No prosthesis,Percutaneous coronary intervention,Predialysis,Prostate,Revascularization
condition,,,,,,,
Acute coronary syndrome,0,0,0,8,0,0,31
Cancer,67,0,0,0,0,41,0
Lower-limb amputation,0,0,10,0,0,0,0
Renal insufficiency,0,1,0,0,9,0,0



✅ Identified 4 real medical conditions
✅ Identified 7 condition subtypes


## Step 2: Basic Data Cleaning

In [6]:
# Basic data cleaning
print("🧹 STEP 2: Basic Data Cleaning")
print("="*50)

df_clean = df_test.copy()
print(f"Starting with: {df_clean.shape}")

# 1. Check for missing values
missing_summary = df_clean.isnull().sum()
missing_cols = missing_summary[missing_summary > 0]

if len(missing_cols) > 0:
    print(f"\n📊 Missing Values Found:")
    for col, count in missing_cols.items():
        pct = (count / len(df_clean)) * 100
        print(f"   • {col}: {count} ({pct:.1f}%)")
    
    # Simple imputation
    for col in missing_cols.index:
        if df_clean[col].dtype in ['int64', 'float64']:
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            print(f"   ✅ Imputed {col} with median: {median_val}")
        else:
            mode_val = df_clean[col].mode()[0] if len(df_clean[col].mode()) > 0 else 'Unknown'
            df_clean[col] = df_clean[col].fillna(mode_val)
            print(f"   ✅ Imputed {col} with mode: {mode_val}")
else:
    print("✅ No missing values found")

# 2. Check for duplicates
duplicates = df_clean.duplicated().sum()
if duplicates > 0:
    print(f"\n⚠️  Found {duplicates} duplicate rows - removing...")
    df_clean = df_clean.drop_duplicates()
else:
    print(f"\n✅ No duplicate rows found")

print(f"\n✅ Cleaned data shape: {df_clean.shape}")

🧹 STEP 2: Basic Data Cleaning
Starting with: (43, 10)

📊 Missing Values Found:
   • age: 1 (2.3%)
   • condition_type: 1 (2.3%)
   • bdi_ii_baseline: 1 (2.3%)
   • bdi_ii_after_intervention_12w: 43 (100.0%)
   • bdi_ii_follow_up_24w: 43 (100.0%)
   ✅ Imputed age with median: 70.0
   ✅ Imputed condition_type with mode: Revascularization
   ✅ Imputed bdi_ii_baseline with median: 10.5
   ✅ Imputed bdi_ii_after_intervention_12w with median: nan
   ✅ Imputed bdi_ii_follow_up_24w with median: nan

✅ No duplicate rows found

✅ Cleaned data shape: (43, 10)


## Step 3: Create Binary Medical Condition Columns

In [7]:
# Create binary columns for ALL real medical conditions
print("🏥 STEP 3: Creating Binary Medical Condition Columns")
print("="*50)

df_features = df_clean.copy()
new_features_log = []
binary_condition_columns = []

# Create binary columns for each real medical condition
print(f"📊 Creating binary columns for {len(real_medical_conditions)} medical conditions...")

for condition in real_medical_conditions:
    # Create clean column name
    col_name = f"condition_{condition.lower().replace(' ', '_').replace('-', '_')}"
    
    # Create binary indicator
    df_features[col_name] = (df_features['condition'] == condition).astype(int)
    new_features_log.append(col_name)
    binary_condition_columns.append(col_name)
    
    # Calculate and display prevalence
    prevalence = df_features[col_name].sum()
    prevalence_pct = (prevalence / len(df_features)) * 100
    print(f"   ✅ {col_name}: {prevalence} patients ({prevalence_pct:.1f}%)")

print(f"\n📊 Created {len(binary_condition_columns)} binary medical condition columns")
print(f"📋 Binary condition columns: {binary_condition_columns}")

🏥 STEP 3: Creating Binary Medical Condition Columns
📊 Creating binary columns for 4 medical conditions...
   ✅ condition_cancer: 19 patients (44.2%)
   ✅ condition_acute_coronary_syndrome: 13 patients (30.2%)
   ✅ condition_renal_insufficiency: 9 patients (20.9%)
   ✅ condition_lower_limb_amputation: 2 patients (4.7%)

📊 Created 4 binary medical condition columns
📋 Binary condition columns: ['condition_cancer', 'condition_acute_coronary_syndrome', 'condition_renal_insufficiency', 'condition_lower_limb_amputation']


## Step 4: Create One-Hot Encoding for Condition Types

In [8]:
# Create one-hot encoding for condition types
print("🏷️  STEP 4: Creating One-Hot Encoding for Condition Types")
print("="*50)

condition_type_columns = []

print(f"📊 Creating one-hot encoding for {len(real_condition_types)} condition types...")

for ctype in real_condition_types:
    # Create clean column name
    clean_name = ctype.lower().replace(' ', '_').replace('-', '_')
    col_name = f"condition_type_{clean_name}"
    
    # Create binary indicator
    df_features[col_name] = (df_features['condition_type'] == ctype).astype(int)
    new_features_log.append(col_name)
    condition_type_columns.append(col_name)
    
    # Calculate and display prevalence
    count = df_features[col_name].sum()
    pct = (count / len(df_features)) * 100
    print(f"   ✅ {col_name}: {count} patients ({pct:.1f}%)")

print(f"\n📊 Created {len(condition_type_columns)} one-hot encoded condition type columns")
print(f"📋 Condition type columns: {condition_type_columns}")

🏷️  STEP 4: Creating One-Hot Encoding for Condition Types
📊 Creating one-hot encoding for 7 condition types...
   ✅ condition_type_breast: 11 patients (25.6%)
   ✅ condition_type_prostate: 8 patients (18.6%)
   ✅ condition_type_revascularization: 13 patients (30.2%)
   ✅ condition_type_no_prosthesis: 2 patients (4.7%)
   ✅ condition_type_predialysis: 8 patients (18.6%)
   ✅ condition_type_percutaneous_coronary_intervention: 1 patients (2.3%)
   ✅ condition_type_dialysis: 0 patients (0.0%)

📊 Created 7 one-hot encoded condition type columns
📋 Condition type columns: ['condition_type_breast', 'condition_type_prostate', 'condition_type_revascularization', 'condition_type_no_prosthesis', 'condition_type_predialysis', 'condition_type_percutaneous_coronary_intervention', 'condition_type_dialysis']


## Step 5: Simple Feature Transformations

In [9]:
# Simple feature transformations
print("🔧 STEP 5: Simple Feature Transformations")
print("="*50)

# 1. BDI Score Transformations
print("\n📊 Creating BDI-related features...")
if 'bdi_ii_baseline' in df_features.columns:
    # Log transformation
    df_features['bdi_baseline_log'] = np.log1p(df_features['bdi_ii_baseline'])
    new_features_log.append('bdi_baseline_log')
    
    # Severity categories - CREATE NUMERIC VERSION with NaN handling
    df_features['bdi_severity_category'] = pd.cut(df_features['bdi_ii_baseline'], 
                                                 bins=[0, 13, 19, 28, 63], 
                                                 labels=[0, 1, 2, 3])  # ✅ NUMERIC LABELS
    # Fill any NaN values with median category (1 = Mild)
    df_features['bdi_severity_category'] = df_features['bdi_severity_category'].fillna(1).astype(int)
    new_features_log.append('bdi_severity_category')
    
    # Squared term
    df_features['bdi_baseline_squared'] = df_features['bdi_ii_baseline'] ** 2
    new_features_log.append('bdi_baseline_squared')
    
    print(f"   ✅ Created 3 BDI-related features (severity: 0=Minimal, 1=Mild, 2=Moderate, 3=Severe)")

# 2. Age-related features
print("\n👥 Creating demographic features...")
if 'age' in df_features.columns:
    # Age groups - CREATE NUMERIC VERSION with NaN handling
    df_features['age_group'] = pd.cut(df_features['age'], 
                                     bins=[0, 30, 45, 60, 100], 
                                     labels=[0, 1, 2, 3])  # ✅ NUMERIC LABELS
    # Fill any NaN values with median category (1 = Middle)
    df_features['age_group'] = df_features['age_group'].fillna(1).astype(int)
    new_features_log.append('age_group')
    
    # Age squared
    df_features['age_squared'] = df_features['age'] ** 2
    new_features_log.append('age_squared')
    
    print(f"   ✅ Created 2 age-related features (age_group: 0=Young, 1=Middle, 2=Mature, 3=Senior)")

# 3. Therapy completion features
print("\n💊 Creating therapy-related features...")
therapy_cols = ['mindfulness_therapies_started', 'mindfulness_therapies_completed']
if all(col in df_features.columns for col in therapy_cols):
    # Completion rate
    df_features['therapy_completion_rate'] = (
        df_features['mindfulness_therapies_completed'] / 
        (df_features['mindfulness_therapies_started'] + 1e-8)
    )
    new_features_log.append('therapy_completion_rate')
    
    # Engagement level - CREATE NUMERIC VERSION
    df_features['therapy_engagement'] = df_features['therapy_completion_rate'].apply(
        lambda x: 2 if x >= 0.8 else (1 if x >= 0.5 else 0)  # ✅ NUMERIC: 2=High, 1=Medium, 0=Low
    )
    new_features_log.append('therapy_engagement')
    
    print(f"   ✅ Created 2 therapy-related features (engagement: 0=Low, 1=Medium, 2=High)")

# 4. Encode categorical variables - PROPER NUMERIC ENCODING
print("\n🏷️  Encoding categorical variables...")
if 'sex' in df_features.columns:
    # Create numeric encoding for sex with missing value handling
    df_features['sex_encoded'] = df_features['sex'].map({'male': 0, 'female': 1})
    # Fill any missing values with mode (0 = male)
    df_features['sex_encoded'] = df_features['sex_encoded'].fillna(0).astype(int)
    new_features_log.append('sex_encoded')
    print(f"   ✅ Encoded sex → sex_encoded (0=male, 1=female)")

print(f"\n✅ Feature transformation completed - ALL FEATURES ARE NUMERIC!")

🔧 STEP 5: Simple Feature Transformations

📊 Creating BDI-related features...
   ✅ Created 3 BDI-related features (severity: 0=Minimal, 1=Mild, 2=Moderate, 3=Severe)

👥 Creating demographic features...
   ✅ Created 2 age-related features (age_group: 0=Young, 1=Middle, 2=Mature, 3=Senior)

💊 Creating therapy-related features...
   ✅ Created 2 therapy-related features (engagement: 0=Low, 1=Medium, 2=High)

🏷️  Encoding categorical variables...
   ✅ Encoded sex → sex_encoded (0=male, 1=female)

✅ Feature transformation completed - ALL FEATURES ARE NUMERIC!


## Step 6: Create Disease Burden Metrics

In [10]:
# Create disease burden metrics using real medical conditions
print("📊 STEP 6: Creating Disease Burden Metrics")
print("="*50)

# Calculate disease burden using binary condition columns
real_condition_sum = df_features[binary_condition_columns].sum(axis=1)
df_features['real_disease_burden_count'] = real_condition_sum
new_features_log.append('real_disease_burden_count')

# Check if disease burden category would be useful
burden_distribution = real_condition_sum.value_counts().sort_index()
print(f"📊 Disease Burden Count Distribution:")
for count, patients in burden_distribution.items():
    pct = (patients / len(df_features)) * 100
    print(f"   • {count} condition(s): {patients} patients ({pct:.1f}%)")

# Only create categorical version if there's variation
unique_counts = len(burden_distribution)
if unique_counts > 1:
    # Disease burden categories - CREATE NUMERIC VERSION
    df_features['real_disease_burden_category'] = real_condition_sum.apply(
        lambda x: 0 if x == 0 else (1 if x == 1 else 2)  # ✅ NUMERIC: 0=None, 1=Single, 2=Multiple
    )
    new_features_log.append('real_disease_burden_category')
    print(f"✅ Created disease burden category (variation exists)")
else:
    print(f"⚠️ Skipping disease burden category - all patients have same burden count ({burden_distribution.index[0]})")

# Create condition type burden (how many different subtypes)
condition_type_sum = df_features[condition_type_columns].sum(axis=1)
df_features['condition_subtype_count'] = condition_type_sum
new_features_log.append('condition_subtype_count')

print(f"\n📊 Condition Subtype Distribution:")
subtype_dist = df_features['condition_subtype_count'].value_counts().sort_index()
for count, patients in subtype_dist.items():
    pct = (patients / len(df_features)) * 100
    print(f"   • {count} subtype(s): {patients} patients ({pct:.1f}%)")

print(f"\n✅ Created useful disease burden metrics - ALL NUMERIC!")
print(f"   • real_disease_burden_count: Integer count (shows variation)")
print(f"   • condition_subtype_count: Integer count (shows variation)")
if unique_counts > 1:
    print(f"   • real_disease_burden_category: 0=None, 1=Single, 2=Multiple")
else:
    print(f"   • Skipped real_disease_burden_category (no variation)")

📊 STEP 6: Creating Disease Burden Metrics
📊 Disease Burden Count Distribution:
   • 1 condition(s): 43 patients (100.0%)
⚠️ Skipping disease burden category - all patients have same burden count (1)

📊 Condition Subtype Distribution:
   • 1 subtype(s): 43 patients (100.0%)

✅ Created useful disease burden metrics - ALL NUMERIC!
   • real_disease_burden_count: Integer count (shows variation)
   • condition_subtype_count: Integer count (shows variation)
   • Skipped real_disease_burden_category (no variation)


## Step 7: Final Dataset Creation and Validation

In [11]:
# Create final dataset
print("📊 STEP 7: Final Dataset Creation and Validation")
print("="*50)

# Define target columns
target_columns = ['bdi_ii_after_intervention_12w', 'bdi_ii_follow_up_24w']
target_columns = [col for col in target_columns if col in df_features.columns]

# Define feature columns (exclude targets and original categorical columns)
exclude_columns = target_columns + ['condition', 'condition_type', 'sex']  # ✅ EXCLUDE ORIGINAL CATEGORICAL
if 'Unnamed: 0' in df_features.columns:
    exclude_columns.append('Unnamed: 0')

feature_columns = [col for col in df_features.columns if col not in exclude_columns]

# Create final dataset
final_columns = feature_columns + target_columns
df_final = df_features[final_columns].copy()

print(f"📊 Final Dataset Summary:")
print(f"   • Total samples: {len(df_final)}")
print(f"   • Feature columns: {len(feature_columns)}")
print(f"   • Target columns: {len(target_columns)}")
print(f"   • Real medical condition columns: {len(binary_condition_columns)}")
print(f"   • Condition type columns: {len(condition_type_columns)}")
print(f"   • Total engineered features: {len(new_features_log)}")

# ✅ CRITICAL VALIDATION: Ensure ALL features are numeric
print(f"\n🔍 Final Validation Checks:")

missing_count = df_final.isnull().sum().sum()
print(f"   • Missing values: {missing_count} {'✅' if missing_count == 0 else '⚠️'}")

inf_count = np.isinf(df_final.select_dtypes(include=[np.number])).sum().sum()
print(f"   • Infinite values: {inf_count} {'✅' if inf_count == 0 else '⚠️'}")

# Check that we have all expected condition columns
expected_conditions = ['condition_cancer', 'condition_acute_coronary_syndrome', 
                      'condition_renal_insufficiency', 'condition_lower_limb_amputation']
missing_conditions = [col for col in expected_conditions if col not in df_final.columns]
print(f"   • All expected conditions present: {'✅' if len(missing_conditions) == 0 else '⚠️'}")
if missing_conditions:
    print(f"     Missing: {missing_conditions}")

# ✅ CRITICAL: Check data types
print(f"\n🔍 Data Type Validation:")
feature_dtypes = df_final[feature_columns].dtypes
non_numeric_features = []
for col, dtype in feature_dtypes.items():
    if dtype == 'object' or str(dtype).startswith('category'):
        non_numeric_features.append(f"{col}: {dtype}")

if len(non_numeric_features) == 0:
    print(f"   ✅ ALL {len(feature_columns)} features are numeric!")
else:
    print(f"   ⚠️ Found {len(non_numeric_features)} non-numeric features:")
    for feature_info in non_numeric_features:
        print(f"     • {feature_info}")

print(f"\n📋 Medical Condition Features ({len(binary_condition_columns)}):")
for i, feature in enumerate(binary_condition_columns, 1):
    prevalence = df_final[feature].sum()
    pct = (prevalence / len(df_final)) * 100
    print(f"   {i:2d}. {feature}: {prevalence} patients ({pct:.1f}%)")

print(f"\n📋 Condition Type Features ({len(condition_type_columns)}):")
for i, feature in enumerate(condition_type_columns, 1):
    count = df_final[feature].sum()
    pct = (count / len(df_final)) * 100
    print(f"   {i:2d}. {feature}: {count} patients ({pct:.1f}%)")

print(f"\n📊 Sample of Final Dataset:")
display(df_final.head())

print(f"\n📊 Data Types Summary:")
print(df_final.dtypes.value_counts())

print(f"\n✅ Final dataset creation completed successfully!")
print(f"✅ ALL FEATURES ARE MACHINE LEARNING READY!")

📊 STEP 7: Final Dataset Creation and Validation
📊 Final Dataset Summary:
   • Total samples: 43
   • Feature columns: 26
   • Target columns: 2
   • Real medical condition columns: 4
   • Condition type columns: 7
   • Total engineered features: 21

🔍 Final Validation Checks:
   • Missing values: 86 ⚠️
   • Infinite values: 0 ✅
   • All expected conditions present: ✅

🔍 Data Type Validation:
   ✅ ALL 26 features are numeric!

📋 Medical Condition Features (4):
    1. condition_cancer: 19 patients (44.2%)
    2. condition_acute_coronary_syndrome: 13 patients (30.2%)
    3. condition_renal_insufficiency: 9 patients (20.9%)
    4. condition_lower_limb_amputation: 2 patients (4.7%)

📋 Condition Type Features (7):
    1. condition_type_breast: 11 patients (25.6%)
    2. condition_type_prostate: 8 patients (18.6%)
    3. condition_type_revascularization: 13 patients (30.2%)
    4. condition_type_no_prosthesis: 2 patients (4.7%)
    5. condition_type_predialysis: 8 patients (18.6%)
    6. cond

,age,hospital_center_id,bdi_ii_baseline,mindfulness_therapies_started,mindfulness_therapies_completed,condition_cancer,condition_acute_coronary_syndrome,condition_renal_insufficiency,condition_lower_limb_amputation,condition_type_breast,condition_type_prostate,condition_type_revascularization,condition_type_no_prosthesis,condition_type_predialysis,condition_type_percutaneous_coronary_intervention,condition_type_dialysis,bdi_baseline_log,bdi_severity_category,bdi_baseline_squared,age_group,age_squared,therapy_completion_rate,therapy_engagement,sex_encoded,real_disease_burden_count,condition_subtype_count,bdi_ii_after_intervention_12w,bdi_ii_follow_up_24w
0,73.0,1,11.0,0,0,1,0,0,0,1,0,0,0,0,0,0,2.484907,0,121.0,3,5329.0,0.000000,0,1,1,1,NaN,NaN
1,67.0,1,25.0,5,1,1,0,0,0,1,0,0,0,0,0,0,3.258097,2,625.0,3,4489.0,0.200000,0,1,1,1,NaN,NaN
2,73.0,1,24.0,23,14,1,0,0,0,1,0,0,0,0,0,0,3.218876,2,576.0,3,5329.0,0.608696,1,1,1,1,NaN,NaN
3,79.0,1,10.0,0,0,1,0,0,0,1,0,0,0,0,0,0,2.397895,0,100.0,3,6241.0,0.000000,0,1,1,1,NaN,NaN
4,80.0,1,24.0,0,0,1,0,0,0,1,0,0,0,0,0,0,3.218876,2,576.0,3,6400.0,0.000000,0,1,1,1,NaN,NaN



📊 Data Types Summary:
int32      14
float64     8
int64       6
Name: count, dtype: int64

✅ Final dataset creation completed successfully!
✅ ALL FEATURES ARE MACHINE LEARNING READY!


In [12]:
# Remove specified disease burden columns from df_final and update related metadata lists
cols_to_remove = ['real_disease_burden_count', 'condition_subtype_count']

print("🔧 Removing columns from df_final if present...")
for c in cols_to_remove:
    if c in df_final.columns:
        df_final.drop(columns=c, inplace=True)
        print(f"   ✅ Dropped: {c}")
    else:
        print(f"   ⚠️ Not found (skipping): {c}")

# Update feature_columns, final_columns and new_features_log if they exist
if 'feature_columns' in globals():
    feature_columns = [c for c in feature_columns if c not in cols_to_remove]
    print(f"   • feature_columns updated ({len(feature_columns)} features)")

if 'final_columns' in globals():
    final_columns = [c for c in final_columns if c not in cols_to_remove]
    print(f"   • final_columns updated ({len(final_columns)} columns)")

if 'new_features_log' in globals():
    new_features_log = [c for c in new_features_log if c not in cols_to_remove]
    print(f"   • new_features_log updated ({len(new_features_log)} entries)")

# Update feature_documentation entries if present
if 'feature_documentation' in globals() and isinstance(feature_documentation, dict):
    # Remove from feature_categories['disease_burden'] if present
    fc = feature_documentation.get('feature_categories', {})
    if 'disease_burden' in fc:
        fc['disease_burden'] = [c for c in fc['disease_burden'] if c not in cols_to_remove]
        feature_documentation['feature_categories'] = fc
    # Update dataset_info totals if present
    if 'dataset_info' in feature_documentation:
        # total_features should reflect current feature_columns length if available
        if 'total_features' in feature_documentation['dataset_info'] and 'feature_columns' in globals():
            feature_documentation['dataset_info']['total_features'] = len(feature_columns)
        # final_shape update
        try:
            feature_documentation['dataset_info']['final_shape'] = df_final.shape
        except Exception:
            pass
    # Remove entries in engineered_features if present
    if 'engineered_features' in feature_documentation:
        ef = feature_documentation['engineered_features']
        if 'columns' in ef:
            ef['columns'] = [c for c in ef['columns'] if c not in cols_to_remove]
            ef['count'] = len(ef['columns'])
            feature_documentation['engineered_features'] = ef
    print("   • feature_documentation updated")

# Final sanity check
print(f"\n📋 df_final shape: {df_final.shape}")
print("📋 Remaining columns (sample):", df_final.columns.tolist()[:20])

🔧 Removing columns from df_final if present...
   ✅ Dropped: real_disease_burden_count
   ✅ Dropped: condition_subtype_count
   • feature_columns updated (24 features)
   • final_columns updated (26 columns)
   • new_features_log updated (19 entries)

📋 df_final shape: (43, 26)
📋 Remaining columns (sample): ['age', 'hospital_center_id', 'bdi_ii_baseline', 'mindfulness_therapies_started', 'mindfulness_therapies_completed', 'condition_cancer', 'condition_acute_coronary_syndrome', 'condition_renal_insufficiency', 'condition_lower_limb_amputation', 'condition_type_breast', 'condition_type_prostate', 'condition_type_revascularization', 'condition_type_no_prosthesis', 'condition_type_predialysis', 'condition_type_percutaneous_coronary_intervention', 'condition_type_dialysis', 'bdi_baseline_log', 'bdi_severity_category', 'bdi_baseline_squared', 'age_group']


## Step 8: Save Dataset and Documentation

In [13]:
# Save the corrected dataset
print("💾 STEP 8: Saving Corrected Dataset and Documentation")
print("="*50)

# Create output directory
output_dir = Path('../../Track1_Data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save final training dataset
final_train_path = output_dir / 'test_corrected_features.xlsx'
df_final.to_excel(final_train_path, index=False)
print(f"✅ Saved corrected training dataset: {final_train_path}")

# Create comprehensive documentation
feature_documentation = {
    'creation_info': {
        'created_date': datetime.now().isoformat(),
        'created_by': 'Corrected Simple Feature Engineering Pipeline',
        'approach': 'Complete medical condition encoding with proper one-hot encoding',
        'key_improvement': 'Includes ALL 4 real medical conditions + proper condition type encoding'
    },
    'dataset_info': {
        'total_samples': len(df_final),
        'total_features': len(feature_columns),
        'total_targets': len(target_columns),
        'final_shape': df_final.shape
    },
    'real_medical_conditions': {
        'count': len(binary_condition_columns),
        'columns': binary_condition_columns,
        'prevalence': {col: int(df_final[col].sum()) for col in binary_condition_columns},
        'note': 'ALL 4 real medical diagnoses properly encoded as binary columns'
    },
    'condition_types': {
        'count': len(condition_type_columns),
        'columns': condition_type_columns,
        'prevalence': {col: int(df_final[col].sum()) for col in condition_type_columns},
        'note': 'Proper one-hot encoding for all condition subtypes'
    },
    'engineered_features': {
        'count': len(new_features_log),
        'columns': new_features_log,
        'note': 'All engineered features with clear non-medical naming'
    },
    'feature_categories': {
        'medical_conditions': binary_condition_columns,
        'condition_subtypes': condition_type_columns,
        'demographic': [f for f in feature_columns if any(x in f.lower() for x in ['age', 'sex'])],
        'psychological': [f for f in feature_columns if 'bdi' in f.lower()],
        'treatment': [f for f in feature_columns if any(x in f.lower() for x in ['therapy', 'mindfulness'])],
        'disease_burden': [f for f in feature_columns if 'burden' in f.lower() or 'subtype_count' in f.lower()],
        'administrative': [f for f in feature_columns if 'hospital' in f.lower()]
    },
    'target_variables': target_columns,
    'quality_checks': {
        'missing_values': int(missing_count),
        'infinite_values': int(inf_count),
        'all_conditions_present': len(missing_conditions) == 0,
        'data_quality_score': 100 if missing_count == 0 and inf_count == 0 and len(missing_conditions) == 0 else 85
    }
}

# Save documentation (use UTF-8 to ensure emojis and other unicode characters are supported)
docs_path = output_dir / 'corrected_feature_documentation.json'
with open(docs_path, 'w', encoding='utf-8') as f:
    json.dump(feature_documentation, f, indent=2, default=str)
print(f"✅ Saved feature documentation: {docs_path}")

# Create summary README
readme_content = f"""# Corrected Feature Engineering Results

## Dataset Overview
- **Created**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Samples**: {len(df_final):,}
- **Features**: {len(feature_columns)}
- **Targets**: {len(target_columns)}

## Key Improvements Over Previous Version
✅ **ALL 4 real medical conditions**: Cancer, Acute coronary syndrome, Renal insufficiency, Lower-limb amputation  
✅ **Complete condition type encoding**: All 7 condition subtypes properly one-hot encoded  
✅ **No missing medical conditions**: Previous version was missing Cancer (64.7% of patients!)  
✅ **Proper disease burden calculation**: Based on all real medical conditions  

## Real Medical Conditions ({len(binary_condition_columns)} total)
{chr(10).join(f'- {col.replace("condition_", "").replace("_", " ").title()}: {df_final[col].sum()} patients ({(df_final[col].sum()/len(df_final)*100):.1f}%)' for col in binary_condition_columns)}

## Condition Subtypes ({len(condition_type_columns)} total)
{chr(10).join(f'- {col.replace("condition_type_", "").replace("_", " ").title()}: {df_final[col].sum()} patients ({(df_final[col].sum()/len(df_final)*100):.1f}%)' for col in condition_type_columns)}

## Data Quality
- Missing values: {missing_count}
- Infinite values: {inf_count}
- All expected conditions present: {'Yes' if len(missing_conditions) == 0 else 'No'}
- Quality score: {100 if missing_count == 0 and inf_count == 0 and len(missing_conditions) == 0 else 85}/100

## Files Generated
- `train_corrected_features.xlsx` - Complete corrected dataset
- `corrected_feature_documentation.json` - Detailed feature documentation
- `Corrected_Feature_Engineering_README.md` - This summary

## Critical Fix
This version fixes the major issue where **Cancer** (affecting 64.7% of patients) was completely missing from the binary condition columns in previous feature engineering attempts.
"""

readme_path = output_dir / 'Corrected_Feature_Engineering_README.md'
# open with utf-8 encoding to avoid UnicodeEncodeError on Windows cp1252 default
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)
print(f"✅ Saved README: {readme_path}")

print(f"\n🎉 CORRECTED FEATURE ENGINEERING COMPLETED SUCCESSFULLY!")
print(f"📊 Complete dataset ready: {final_train_path}")
print(f"📋 Full documentation: {docs_path}")
print(f"📖 Summary README: {readme_path}")
print(f"\n✅ ALL 4 real medical conditions properly encoded!")
print(f"✅ ALL 7 condition subtypes properly one-hot encoded!")
print(f"✅ No more missing medical conditions in the analysis!")

💾 STEP 8: Saving Corrected Dataset and Documentation
✅ Saved corrected training dataset: ..\..\Track1_Data\processed\test_corrected_features.xlsx
✅ Saved feature documentation: ..\..\Track1_Data\processed\corrected_feature_documentation.json
✅ Saved README: ..\..\Track1_Data\processed\Corrected_Feature_Engineering_README.md

🎉 CORRECTED FEATURE ENGINEERING COMPLETED SUCCESSFULLY!
📊 Complete dataset ready: ..\..\Track1_Data\processed\test_corrected_features.xlsx
📋 Full documentation: ..\..\Track1_Data\processed\corrected_feature_documentation.json
📖 Summary README: ..\..\Track1_Data\processed\Corrected_Feature_Engineering_README.md

✅ ALL 4 real medical conditions properly encoded!
✅ ALL 7 condition subtypes properly one-hot encoded!
✅ No more missing medical conditions in the analysis!


## Summary

### ✅ **What This Corrected Version Provides:**

1. **🏥 Complete Medical Condition Coverage**: 
   - `condition_cancer` (108 patients - 64.7%)
   - `condition_acute_coronary_syndrome` (39 patients - 23.4%)
   - `condition_renal_insufficiency` (10 patients - 6.0%)
   - `condition_lower_limb_amputation` (10 patients - 6.0%)

2. **🏷️ Complete Condition Type Encoding**:
   - `condition_type_breast` (67 patients)
   - `condition_type_prostate` (41 patients)
   - `condition_type_revascularization` (31 patients)
   - And 4 more condition subtypes

3. **📊 Accurate Disease Burden Metrics**:
   - Based on ALL real medical conditions
   - Proper calculation of single vs multiple conditions
   - Condition subtype counting

### 🚨 **Critical Fix Applied:**
Previous feature engineering was **missing Cancer** - the most prevalent condition affecting 64.7% of patients! This has been corrected.

### 🎯 **Ready for Analysis:**
This dataset now provides complete, accurate medical condition information for proper clinical analysis and machine learning model development.